Import packages needed for the scenario.

In [1]:
import mosaik
from mosaik.util import connect_randomly, connect_many_to_one

import data.webvis_data as wv_data

Specify simulator configurations.

In [2]:
sim_config = {
    'PyPower': {
        'python': 'mosaik_pypower.mosaik:PyPower'
    },
    'CSV': {
        'python': 'simulators.ws2017_csv_sim:CSV'
    },
    'DB': {
        'cmd': 'mosaik-hdf5 %(addr)s'
    },
    'WebVis': {
        'cmd': 'mosaik-web -s 0.0.0.0:8000 %(addr)s'
    }
}

Add PV data file.

In [3]:
END = 7 * 24 * 60 * 60 # one week in seconds
START = '2014-01-01 00:00:00'
PROFILE_DATA = 'data/mprofiles.txt' # .data in your setup
GRID_FILE = 'data/demo_lv_grid.json'
PV_DATA = 'data/pv_10kw.txt' # .csv in your setup

Set up the "world" of the scenario.

In [4]:
world = mosaik.World(sim_config)

Initialize the simulators.

In [5]:
pypower = world.start('PyPower', step_size=15*60)
hhsim = world.start('CSV', sim_start=START, datafile=PROFILE_DATA)
pvsim = world.start('CSV', sim_start=START, datafile=PV_DATA)

Starting "PyPower" as "PyPower-0" ...
Starting "CSV" as "CSV-0" ...
Starting "CSV" as "CSV-1" ...


Instantiate model entities.

In [6]:
grid = pypower.Grid(gridfile=GRID_FILE).children
houses = hhsim.House.create(50)

nodes = [element for element in grid if 'node' in element.eid]

pvs = pvsim.PV.create(20)

Connect model entities.

In [7]:
connect_randomly(world, houses, nodes, 'P')
connect_randomly(world, pvs, nodes, 'P')

{Entity('PyPower-0', '0-node_a3', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x0000000010776828>),
 Entity('PyPower-0', '0-node_b10', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x0000000010776828>),
 Entity('PyPower-0', '0-node_b2', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x0000000010776828>),
 Entity('PyPower-0', '0-node_b3', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x0000000010776828>),
 Entity('PyPower-0', '0-node_b4', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x0000000010776828>),
 Entity('PyPower-0', '0-node_b5', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x0000000010776828>),
 Entity('PyPower-0', '0-node_b6', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x0000000010776828>),
 Entity('PyPower-0', '0-node_b8', 'PyPower', PQBus, [], <mosaik.simmanager.LocalProcess object at 0x0000000010776828>),
 Entity('PyPower-0', '0-node_b9', 'PyPo

Initialize, configure and instantiate visualization tool.

In [8]:
webvis = world.start('WebVis', start_date=START, step_size=60)
webvis.set_config(ignore_types=['Topology', 'Grid', 'Database'])
vis_topo = webvis.Topology()

Starting "WebVis" as "WebVis-0" ...


Connect model entities to visualizer.

In [9]:
busses = [elem for elem in grid if elem.type in ('RefBus', 'PQBus')]
connect_many_to_one(world, busses, vis_topo, 'P', 'Vm')
connect_many_to_one(world, houses, vis_topo, 'P')

connect_many_to_one(world, pvs, vis_topo, 'P')

webvis.set_etypes(wv_data.etypes) # Additional configuration of visualization (special to this tool)

Execute co-simulation.

In [10]:
world.run(until=END)

Starting simulation.
Simulation finished successfully.
